# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from Config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('Output_Results/Weather_Dict.csv')
weather_df = pd.DataFrame(weather_data)
weather_df

,Unnamed: 0,City,Country,Temperature,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,0,Punta Arenas,CL,62.29,62.60,63,0,2.30,-53.1500,-70.9167
1,1,Isangel,VU,84.20,84.20,74,75,9.22,-19.5500,169.2667
2,2,Kaitangata,NZ,57.00,57.00,90,100,4.00,-46.2817,169.8464
3,3,Mataura,NZ,59.20,59.20,65,100,9.31,-46.1927,168.8643
4,4,Ilulissat,GL,17.60,17.60,45,0,12.66,69.2167,-51.1000
...,...,...,...,...,...,...,...,...,...,...
543,543,Iralaya,HN,76.96,76.96,80,1,9.80,15.0000,-83.2333
544,544,Sozimskiy,RU,-23.57,-23.57,86,0,3.74,59.7418,52.2482
545,545,Porbandar,IN,55.40,55.40,82,20,3.44,21.6422,69.6093
546,546,Tautira,PF,84.09,84.20,61,75,14.97,-17.7333,-149.1500


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Store the Lat and Lng as Locations and Humidity as the weight
locations= weather_df[['Latitude','Longitude']]
humidity = weather_df['Humidity']

In [5]:
#Create heatmao layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Weather conditions have wind speedo less than 12 mph, temp greater than 77, max temp greater than 80, and cloudiness less than 20%
weather_conditions = weather_df.loc[(weather_df['Wind Speed'] <= 12) & (weather_df['Temperature'] >=77) & (weather_df['Max Temp'] >=80) & (weather_df['Cloudiness'] <=20)]
weather_conditions.dropna()

,Unnamed: 0,City,Country,Temperature,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
5,5,Phan Thiet,VN,81.68,81.68,63,20,3.71,10.9333,108.1000
36,36,Avarua,CK,86.00,86.00,70,20,9.22,-21.2078,-159.7750
86,86,George Town,MY,82.06,84.20,74,20,5.75,5.4112,100.3354
104,104,Adiaké,CI,80.60,80.60,100,20,5.75,5.2863,-3.3040
120,120,Codrington,AU,82.44,82.44,35,0,4.09,-38.2667,141.9667
175,175,Tra Vinh,VN,83.30,83.30,64,6,9.95,9.9347,106.3453
198,198,Kavaratti,IN,81.10,81.10,70,0,11.95,10.5669,72.6420
214,214,Dingle,PH,82.74,82.74,78,13,4.74,10.9995,122.6711
215,215,Mount Gambier,AU,87.80,87.80,22,0,5.75,-37.8333,140.7667
243,243,Hervey Bay,AU,81.00,81.00,56,19,8.99,-25.2986,152.8535


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df
hotel_df = weather_conditions[['City','Country','Latitude','Longitude']]
hotel_df['Hotel Name']=""
hotel_df


<ipython-input-7-7bc791d3244d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""


,City,Country,Latitude,Longitude,Hotel Name
5,Phan Thiet,VN,10.9333,108.1000,
36,Avarua,CK,-21.2078,-159.7750,
86,George Town,MY,5.4112,100.3354,
104,Adiaké,CI,5.2863,-3.3040,
120,Codrington,AU,-38.2667,141.9667,
175,Tra Vinh,VN,9.9347,106.3453,
198,Kavaratti,IN,10.5669,72.6420,
214,Dingle,PH,10.9995,122.6711,
215,Mount Gambier,AU,-37.8333,140.7667,
243,Hervey Bay,AU,-25.2986,152.8535,


In [8]:
#set parameters and base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
         "keyword":"hotel",
         "radius": 5000,
         "key": g_key}

In [9]:
#Set loop to search hotels 
#Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    city = row["City"]
    country = row["Country"]
    
    #API request, add params & print results
    print(f'Searching for Hotels by Index ({index}): {city}.')
    params["location"]=f"{lat}, {lon}"

    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    #save hotel information
    try:
        hotelname= response['results'][0]['name']
        print(f'The closest hotel to {city} is {hotelname}.')
        hotel_df.loc[index,"Hotel Name"] = hotelname
        
    except:
        print(f"Missing Hotel Result, searching for next record..")
        
    print('--------------------------')
    

Searching for Hotels by Index (5): Phan Thiet.
The closest hotel to Phan Thiet is Yurii Hotel.
--------------------------
Searching for Hotels by Index (36): Avarua.


C:\Users\JNemec1\anaconda\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


The closest hotel to Avarua is Sunrise Beach Bungalows.
--------------------------
Searching for Hotels by Index (86): George Town.
The closest hotel to George Town is Eastern & Oriental Hotel, Penang.
--------------------------
Searching for Hotels by Index (104): Adiaké.
Missing Hotel Result, searching for next record..
--------------------------
Searching for Hotels by Index (120): Codrington.
The closest hotel to Codrington is Codrington Gardens Bed And Breakfast.
--------------------------
Searching for Hotels by Index (175): Tra Vinh.
The closest hotel to Tra Vinh is Cuu Long Hotel.
--------------------------
Searching for Hotels by Index (198): Kavaratti.
The closest hotel to Kavaratti is Bismillah Hotel And Lodging.
--------------------------
Searching for Hotels by Index (214): Dingle.
Missing Hotel Result, searching for next record..
--------------------------
Searching for Hotels by Index (215): Mount Gambier.
The closest hotel to Mount Gambier is Commodore on the Park.
----

In [10]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
5,Phan Thiet,VN,10.9333,108.1000,Yurii Hotel
36,Avarua,CK,-21.2078,-159.7750,Sunrise Beach Bungalows
86,George Town,MY,5.4112,100.3354,"Eastern & Oriental Hotel, Penang"
104,Adiaké,CI,5.2863,-3.3040,
120,Codrington,AU,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
175,Tra Vinh,VN,9.9347,106.3453,Cuu Long Hotel
198,Kavaratti,IN,10.5669,72.6420,Bismillah Hotel And Lodging
214,Dingle,PH,10.9995,122.6711,
215,Mount Gambier,AU,-37.8333,140.7667,Commodore on the Park
243,Hervey Bay,AU,-25.2986,152.8535,Oaks Hervey Bay Resort and Spa


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))